In [ ]:
%pip install langchain gpt4all bs4

In [27]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import LlamaCppEmbeddings

from gpt4all import GPT4All, Embed4All

import pickle
import faiss

In [28]:
def get_subpages(base_url):
    # Initialize an empty list to store the subpage URLs
    subpages = []

    # Send a request to the base_url to get the HTML content
    response = requests.get(base_url)
    if response.status_code != 200:
        raise ValueError(f"Failed to fetch the page. Status code: {response.status_code}")

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all anchor tags (<a>) in the HTML
    anchor_tags = soup.find_all('a')

    # Extract the href attribute from each anchor tag and form the absolute URLs
    for tag in anchor_tags:
        href = tag.get('href')
        if href:
            subpage_url = urljoin(base_url, href)
            subpages.append(subpage_url)

    # Return the list of subpage URLs
    return subpages


In [19]:
urls = 'https://github.wpilib.org/allwpilib/docs/release/java/'

In [ ]:
subpages_list = get_subpages(urls)
s_list = []
for page in subpages_list:
    if page.startswith("https://github.wpilib.org/allwpilib/docs/release/java/edu/wpi/first"):
        s_list.append(page)
s_list

In [6]:
example_page = ['https://github.wpilib.org/allwpilib/docs/release/java/edu/wpi/first/apriltag/AprilTagDetection.html']
loaders = UnstructuredURLLoader(urls=example_page)
data = loaders.load()
text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)

docs = text_splitter.split_documents(data)

In [32]:
llama = LlamaCppEmbeddings(model_path='./ggml-model-q4_0.bin')
VectorStore_OpenAI = FAISS.from_documents(docs, embedder)

with open("faiss_store_openai.pkl", "wb") as f:
  pickle.dump(VectorStore_OpenAI, f)
with open("faiss_store_openai.pkl", "rb") as f:
  VectorStore = pickle.load(f)

ModuleNotFoundError: Could not import llama-cpp-python library. Please install the llama-cpp-python library to use this embedding model: pip install llama-cpp-python